# Comparison of treatment and control sites

In [1]:
import ee, eemont
import geemap
import pandas as pd
import geopandas as gpd
from bisonlab.data import landsat_7_sr, landsat_8_sr, landsat_9_sr, s2_sr_harmonized, s2
from bisonlab.io import kml_to_geodataframe, kmz_to_geodataframe
from bisonlab.data import LOCAL_DATA_DIR, DATA_DIR
from bisonlab.utils import mask_include, mask_exclude
from pathlib import Path
import datetime as dt
import numpy as np
import random
random.seed(123)
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import geemap.colormaps as cm
from math import ceil, floor

In [2]:
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


In [3]:
crs = "EPSG:5070"

# Parcels

In [16]:
# filepath = DATA_DIR / "Buffalo Expansion.kmz"
# df_parcels = kmz_to_geodataframe(filepath)

# df_parcels.Name.unique()

In [26]:
# Open buffalo expansion geometries
filepath = DATA_DIR / "Buffalo Expansion.kmz"
df_parcels = kmz_to_geodataframe(filepath)
df_parcels = df_parcels.drop(columns="Description")
df_parcels = df_parcels.rename(columns={"Name": "name"})

# Select subset of parcels
parcels_of_interest = [
    "Shoshone Tribe",
    "Hellyer Place",
    "Hellyer Tribal Lease",
    "Buffalo Initiative",
    "Wickstram Property",
    "Hoopengarner Property",
    "Adels Property",
]
df_parcels = df_parcels.loc[df_parcels.name.isin(parcels_of_interest)].reset_index(drop=True)

df_parcels = df_parcels.to_crs(crs=crs)

# Convert parcels dataframe to ee.featureCollection
parcels = geemap.geopandas_to_ee(df_parcels)

parcel_labels = df_parcels.name.to_list()
parcel_colors = ["006400", "ffbb22", "ffff4c", "f096ff", "fa0000", "b4b4b4", "fa0000"]
parcels_styled = geemap.ee_vector_style(parcels, column='name', labels=parcel_labels, fillColor='00000000', color=parcel_colors, width=5)

# Get maximum bounds of all parcels
parcel_bounds = parcels.geometry().dissolve().bounds()
parcel_bounds_shapely = df_parcels.dissolve().geometry.item()

In [18]:
# "006400", "ffbb22", "ffff4c", "f096ff", "fa0000", "b4b4b4", "f0f0f0", "0064c8", "0096a0", "00cf75", "fae6a0"

# Remote Sensing / Soil Data Layers

### Terrain
* Elevation / DEM
    - [USGS_3DEP_1m](https://developers.google.com/earth-engine/datasets/catalog/USGS_3DEP_1m)
    - `1 m` resolution

In [19]:
# Compute terrain properties from DEM
dem = ee.ImageCollection("USGS/3DEP/1m").filterBounds(parcel_bounds)

terrain = dem.map(ee.Terrain.products).mosaic()

elevation = terrain.select('elevation')

# Slope units are degrees, range is [0,90)
slope = terrain.select('slope')

# Aspect units are degrees where 0=N, 90=E, 180=S, 270=W
aspect = terrain.select('aspect')

# Hillshade is calculated based on illumination azimuth=270, elevation=45
hillshade = terrain.select('hillshade')

### Soil taxonomy
* sourced from SSURGO

In [20]:
df_soil_taxonomy = gpd.read_parquet(DATA_DIR / "soil_taxonomy.parquet") 
df_soil_taxonomy = df_soil_taxonomy.to_crs(crs=crs)

# Only include map units that intersect parcels
df_soil_taxonomy = df_soil_taxonomy[df_soil_taxonomy.intersects(parcel_bounds_shapely)]

unique_soil = list(df_soil_taxonomy['MUSYM'].unique())
soil_taxonomy_map = dict(zip(unique_soil, range(1, len(unique_soil) + 1))) 
df_soil_taxonomy["soil_id"] = df_soil_taxonomy.MUSYM.map(soil_taxonomy_map)

soil_taxonomy = geemap.geopandas_to_ee(df_soil_taxonomy)

soil_taxonomy_labels = unique_soil
# soil_taxonomy_colors = [f"{random.randint(0, 0xFFFFFF):06x}" for i in range(len(soil_taxonomy_labels))]
soil_taxonomy_colors = ['2cdbad', 'c1541a', '23c6e9', '03606c', 'a183d6', 'e58a6c', '343e28', '16786a', '2f7ed6', '48cb2b', '40a3cc', '0adf71', '956702', 'dc7c87', 'f42f4d']
soil_taxonomy_styled = geemap.ee_vector_style(
    soil_taxonomy, column='MUSYM', labels=soil_taxonomy_labels, fillColor=soil_taxonomy_colors, color='00000000'
)

In [21]:
# df_soil_taxonomy.to_file('soil_type.kml', driver='KML')

In [22]:
pd.set_option('display.max_colwidth', None)

In [23]:
df_soil_taxonomy.loc[:, ["MUSYM", "desc"]].drop_duplicates()

,MUSYM,desc
5,EuB,"Ethete loam, saline, 0 to 6 percent slopes"
41,CRF,"Clifterson-Rock land association, steep"
72,Cw,"Crowheart loam, 0 to 3 percent slopes, occasionally flooded"
119,EtB,"Ethete loam, 3 to 6 percent slopes"
125,ZZ900,Water
138,GrA,"Griffy loam, 0 to 3 percent slopes"
154,Bg,"Bigwin fine sandy loam, 0 to 2 percent slopes, frequently flooded"
190,FnA,"Fivemile silty clay loam, 0 to 3 percent slopes"
202,PaC,"Pavillion sandy clay loam, 3 to 10 percent slopes"
247,Wa,Wet alluvial land


### Irrigation
* GFSAD Landsat-Derived Global Rainfed and Irrigated-Cropland Product (LGRIP)
    - `https://gee-community-catalog.org/projects/lgrip30/`

In [24]:
irrigation = ee.ImageCollection("projects/sat-io/open-datasets/GFSAD/LGRIP30").filterBounds(parcel_bounds)

# Class Label, Name, Description
# 0 Ocean, Ocean and Water bodies
# 1 Non-croplands, Land with other land use
# 2 Irrigated croplands, Agricultural croplands that are irrigated
# 3 Rainfed croplands, Agricultural croplands that are rainfed

irrigation_colors = ["0050cb","d58855","c2d30c","379a4b"]
irrigation_labels = ["Ocean", "Non-croplands", "Irrigated croplands", "Rainfed croplands"]
irrigation_vis = {"min": 0, "max": 3, "palette": irrigation_colors}

## Plot all remote sensing layers

In [27]:
m = geemap.Map(center=(-108.808, 43.205), zoom=14, height=800, basemap='SATELLITE')

# m.addLayer(slope, {"min": 0, "max": 60}, 'Slope')
# m.addLayer(slope.lt(10), {}, 'Slope < 10 deg')
# m.addLayer(aspect, {"min": 0, "max": 359.99}, 'Aspect')
# m.addLayer(hillshade, {"min": 0, "max": 255}, 'Hillshade')

# m.addLayer(soil_taxonomy, {}, 's')
m.addLayer(soil_taxonomy_styled, {}, 'Soil Taxonomy')
m.add_legend(title="Soil group", labels=soil_taxonomy_labels, colors=soil_taxonomy_colors, position="bottomright")

# m.addLayer(irrigation, irrigation_vis, "Irrigated")
# m.add_legend(title='Irrigation', labels=irrigation_labels, colors=irrigation_colors, position="topleft")

m.addLayer(parcels_styled, {}, "Parcels")
m.add_legend(title='parcels', labels=parcel_labels, colors=parcel_colors, position="bottomleft")
# m.centerObject(parcels, 14)
m

Map(center=[-108.808, 43.205], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [ ]:
# fc = m.draw_features

In [ ]:
# fc[0].geometry().getInfo()

In [ ]:
# _a_bison_7_upper = ee.Geometry.Polygon(coords=[[[-108.79312, 43.206349],
#    [-108.788213, 43.20641],
#    [-108.788719, 43.205807],
#    [-108.789421, 43.204898],
#    [-108.789963, 43.2042],
#    [-108.790124, 43.203932],
#    [-108.791213, 43.204308],
#    [-108.791572, 43.204548],
#    [-108.792396, 43.205366],
#    [-108.7929, 43.206058],
#    [-108.79312, 43.206349]]])

In [ ]:
# m = geemap.Map(height=800, basemap='SATELLITE')
# m.addLayer(geom, {}, "")
# m.centerObject(geom, 14)
# m

In [ ]:
df_soil_taxonomy.loc[:, ["MUSYM", "desc"]].drop_duplicates()

In [ ]:
# Comparison a
_comparison_a_soil_geom = soil_taxonomy.filter(ee.Filter.inList("MUSYM", ["EtA"])).geometry().buffer(-10)

_comparison_a_cattle = ee.Geometry.Polygon(coords=[[[-108.804828, 43.213152],
   [-108.804967, 43.21291],
   [-108.804737, 43.212812],
   [-108.804603, 43.21264],
   [-108.804646, 43.212448],
   [-108.804308, 43.212362],
   [-108.804098, 43.211979],
   [-108.803712, 43.211753],
   [-108.802983, 43.211174],
   [-108.801856, 43.210259],
   [-108.801717, 43.209845],
   [-108.80162, 43.209516],
   [-108.80103, 43.20929],
   [-108.800558, 43.209344],
   [-108.79986, 43.209055],
   [-108.799732, 43.208789],
   [-108.798895, 43.208586],
   [-108.798208, 43.208148],
   [-108.79764, 43.207968],
   [-108.7972, 43.207616],
   [-108.79779, 43.206998],
   [-108.79632, 43.206646],
   [-108.796266, 43.206318],
   [-108.795419, 43.206326],
   [-108.79544, 43.208641],
   [-108.795419, 43.209555],
   [-108.796899, 43.210259],
   [-108.799024, 43.211221],
   [-108.800687, 43.211847],
   [-108.802779, 43.212511],
   [-108.804828, 43.213152]]]
)
comparison_a_cattle = _comparison_a_soil_geom.intersection(_comparison_a_cattle, maxError=0.01)

_comparison_a_bison_7 = ee.Geometry.Polygon(coords=[[[-108.79485, 43.206322],
   [-108.791165, 43.20222],
   [-108.790092, 43.201727],
   [-108.788826, 43.201266],
   [-108.784642, 43.205086],
   [-108.788279, 43.206419],
   [-108.79485, 43.206322]]])
comparison_a_bison_7 = _comparison_a_soil_geom.intersection(_comparison_a_bison_7, maxError=0.01)

_comparison_a_bison_7_1 = ee.Geometry.Polygon(coords=[[[-108.79312, 43.206349],
   [-108.788213, 43.20641],
   [-108.788719, 43.205807],
   [-108.789421, 43.204898],
   [-108.789963, 43.2042],
   [-108.790124, 43.203932],
   [-108.791213, 43.204308],
   [-108.791572, 43.204548],
   [-108.792396, 43.205366],
   [-108.7929, 43.206058],
   [-108.79312, 43.206349]]])

comparison_a_bison_7_1 = _comparison_a_soil_geom.intersection(_comparison_a_bison_7_1, maxError=0.01)

_comparison_a_bison_7_2 = ee.Geometry.Polygon(coords=[[[-108.788133, 43.206341],
   [-108.788469, 43.205995],
   [-108.789837, 43.204247],
   [-108.790038, 43.203895],
   [-108.788576, 43.203391],
   [-108.788442, 43.203373],
   [-108.787978, 43.203321],
   [-108.786975, 43.202924],
   [-108.784644, 43.205066],
   [-108.788133, 43.206341]]]
)
comparison_a_bison_7_2 = _comparison_a_soil_geom.intersection(_comparison_a_bison_7_2, maxError=0.01)

_comparison_a_bison_1 = ee.Geometry.Polygon(coords=[[[-108.784557, 43.204951],
   [-108.788407, 43.20136],
   [-108.777909, 43.199349],
   [-108.776171, 43.200992],
   [-108.78071, 43.203448],
   [-108.781442, 43.203803],
   [-108.784349, 43.204932],
   [-108.784557, 43.204951]]])
comparison_a_bison_1 = _comparison_a_soil_geom.intersection(_comparison_a_bison_1, maxError=0.01)

In [ ]:
# Comparison b
_comparison_b_soil_geom = soil_taxonomy.filter(ee.Filter.inList("MUSYM", ["EtA", "GrA"])).geometry()

_comparison_b_cattle = ee.Geometry.BBox(-108.80535162985326, 43.20626579152538, -108.798369, 43.20653606179568)
comparison_b_cattle = _comparison_b_soil_geom.intersection(_comparison_b_cattle, maxError=0.01)

_comparison_b_bison_7 = ee.Geometry.BBox(-108.80535162985326, 43.20599552125508, -108.798369, 43.20626579152538)
comparison_b_bison_7 = _comparison_b_soil_geom.intersection(_comparison_b_bison_7, maxError=0.01)

In [ ]:
# Comparison c
_comparison_c_soil_geom = soil_taxonomy.filter(ee.Filter.inList("MUSYM", ["GrA"])).geometry().buffer(-10)

_comparison_c_bison_7 = ee.Geometry.Polygon(coords=[[[-108.805305, 43.206236],
   [-108.805262, 43.204961],
   [-108.803766, 43.2045],
   [-108.803551, 43.204554],
   [-108.80221, 43.204331],
   [-108.802065, 43.204066],
   [-108.80111, 43.203608],
   [-108.80074, 43.203444],
   [-108.800193, 43.203115],
   [-108.798482, 43.201547],
   [-108.798235, 43.201467],
   [-108.797983, 43.201578],
   [-108.797792, 43.201901],
   [-108.797516, 43.202005],
   [-108.796913, 43.201926],
   [-108.796728, 43.201819],
   [-108.7961, 43.200771],
   [-108.79591, 43.200343],
   [-108.795132, 43.199181],
   [-108.791071, 43.199207],
   [-108.790977, 43.199289],
   [-108.791135, 43.199502],
   [-108.791312, 43.199696],
   [-108.791403, 43.199825],
   [-108.79165, 43.200278],
   [-108.7918, 43.200423],
   [-108.792525, 43.200624],
   [-108.793539, 43.200925],
   [-108.793919, 43.20127],
   [-108.794713, 43.201973],
   [-108.794807, 43.202079],
   [-108.794855, 43.202171],
   [-108.794898, 43.202531],
   [-108.794982, 43.202732],
   [-108.795727, 43.203579],
   [-108.795955, 43.203729],
   [-108.796467, 43.203921],
   [-108.796768, 43.204077],
   [-108.796948, 43.204216],
   [-108.797514, 43.204801],
   [-108.797696, 43.205053],
   [-108.797841, 43.205326],
   [-108.797996, 43.205821],
   [-108.798476, 43.206226],
   [-108.805305, 43.206236]]])
comparison_c_bison_7 = _comparison_c_soil_geom.intersection(_comparison_c_bison_7, maxError=0.01)

_comparison_c_bison_1 = ee.Geometry.Polygon(coords=[[[-108.787407, 43.200413],
   [-108.787235, 43.199203],
   [-108.787155, 43.198795],
   [-108.786436, 43.197646],
   [-108.78587, 43.196688],
   [-108.785252, 43.19562],
   [-108.780431, 43.19565],
   [-108.775656, 43.195189],
   [-108.772271, 43.194848],
   [-108.772287, 43.195623],
   [-108.772931, 43.195808],
   [-108.773859, 43.196231],
   [-108.774189, 43.196512],
   [-108.774763, 43.196583],
   [-108.775482, 43.19686],
   [-108.775923, 43.197066],
   [-108.776611, 43.197333],
   [-108.777947, 43.197615],
   [-108.777555, 43.198622],
   [-108.781863, 43.199584],
   [-108.779283, 43.196757],
   [-108.779433, 43.196702],
   [-108.782276, 43.199659],
   [-108.787056, 43.200621],
   [-108.787436, 43.200609],
   [-108.787407, 43.200413]]])
comparison_c_bison_1 = _comparison_c_soil_geom.intersection(_comparison_c_bison_1, maxError=0.01)

In [ ]:
_a = kml_to_geodataframe(LOCAL_DATA_DIR / "Bison7_green_side.kml")
_b = kml_to_geodataframe(LOCAL_DATA_DIR / "Bison7-reddish_side.kml")
bison7_green_side = geemap.geopandas_to_ee(_a).first().geometry()
bison7_reddish_side = geemap.geopandas_to_ee(_b).first().geometry()

In [ ]:
site_names = ['a_cattle', 'a_bison_7', 'a_bison_7_1', 'a_bison_7_2', 'a_bison_1', 'b_bison_7', 'b_cattle', 'c_bison_7', 'c_bison_1', 'bison_7_green', 'bison_7_reddish']
site_geom = [comparison_a_cattle, comparison_a_bison_7, comparison_a_bison_7_1, comparison_a_bison_7_2, comparison_a_bison_1, comparison_b_bison_7, comparison_b_cattle, comparison_c_bison_7, comparison_c_bison_1, bison7_green_side, bison7_reddish_side]

In [ ]:
comparison_sites = ee.FeatureCollection([
    ee.Feature(geom, {'name': name})
    for name, geom in zip(site_names, site_geom)
])

In [ ]:
sites_styled = geemap.ee_vector_style(comparison_sites, column='name', color=["ffff4c", "f096ff", "fa0000", "b4b4b4", "f0f0f0", "0064c8", "0096a0", "00cf75", "fae6a0", "0f0f0f", "00fa00"])

In [ ]:
m = geemap.Map(height=800)

m.addLayer(parcels_styled, {}, "Parcels")
m.add_legend(title='parcels', labels=parcel_labels, colors=parcel_colors, position="bottomleft")
m.addLayer(comparison_sites, {}, "comparison_sites")
m.addLayer(sites_styled, {}, "sites")
m.centerObject(parcels, 14)
m

In [ ]:
m = geemap.Map(height=800)
m.addLayer(soil_taxonomy_styled, {}, 'Soil Taxonomy')
m.add_legend(title="Soil group", labels=soil_taxonomy_labels, colors=soil_taxonomy_colors, position="bottomright")

# m.addLayer(parcels_styled, {}, "Parcels")
# m.add_legend(title='parcels', labels=parcel_labels, colors=parcel_colors, position="bottomleft")

# m.addLayer(comparison_sites.filterMetadata("name", "equals", "c_bison_7"), {}, "comparison_sites")
# m.addLayer(soil_taxonomy.filter(ee.Filter.inList("MUSYM", ["Eta", "Cw"])), {}, "f")
m.centerObject(parcels, 14)
m

# Intersect soil properties with parcels 
* Comparison a - define a buffer and stay within the pink - Eta
* Comparison b - across the fence, here we’re lumping the soils Eta and Cw - stay out of CRF.
* Comparison c - stay within GrA, give a small buffer on CRF.


We want only regions with `slope < 10 deg`


In [ ]:
# imgcol = (
#         ee.ImageCollection("COPERNICUS/S2")
#         .filterBounds(aoi)
#         .filterDate(dt.datetime(2017, 1, 1), dt.datetime(2018, 12, 31))
#         .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 20))

In [ ]:
start_date, end_date = dt.datetime(2017, 1, 1), dt.datetime(2018, 12, 31)
aoi = bison7_green_side

imgcol = s2_sr_harmonized(
    aoi, start_date, end_date, cloud_filter=100, cloud_prob_thresh=100
).spectralIndices('NDVI')

In [ ]:
# aoi = bison7_green_side
imgcol = s2(aoi, dt.datetime(2017, 1, 1), dt.datetime(2018, 12, 31), cloud_filter=100).spectralIndices('NDVI')

In [ ]:
def _f(img):
    qa_band = img.select("QA60")
    # Create masks for each QA bit of interest. All bits should be set to zero,
    # indicating clear conditions.
    clear_bit_masks = [qa_band.bitwiseAnd(bit_mask).eq(0) for bit_mask in [1 << 10, 1 << 11]]
    # Note that `ee.Reducer.allNonZero` is preferred over the deprecated `ee.Reducer.and`
    clear_mask = ee.Image.cat(clear_bit_masks).reduce(ee.Reducer.allNonZero())

    return img.updateMask(clear_mask)

In [ ]:
imgcol = imgcol.map(_f)

In [ ]:
ts = imgcol.getTimeSeriesByRegion(
    reducer=[
        ee.Reducer.mean(),
        ee.Reducer.count(),
    ],
    geometry=aoi,
    bands=['NDVI'],
    scale=10,
    dateColumn="date",
    naValue=None,
    )

In [ ]:
df = geemap.ee_to_pandas(ts)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
ndvi = df[df['reducer'].eq('mean')].set_index('date')[['NDVI']]

In [ ]:
ndvi.plot(marker='.', ls='none');

In [ ]:
ts.first().propertyNames().getInfo()

In [ ]:
size = ts.size().getInfo()
chunk_size = 100
n_chunks = ceil(size / chunk_size)

offset = 0
df = geemap.ee_to_pandas(ee.FeatureCollection(ts.toList(count=chunk_size, offset=offset)))

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# df.loc[df.date.dt.year.eq(2019) & df.date.dt.month.eq(1)]

In [ ]:
img1 = imgcol.select('NDVI').closest('2019-01-08').first()

In [ ]:
img2 = imgcol.select('NDVI').closest('2019-01-25').first()

In [ ]:
m = geemap.Map(height=800)

vis_params = {'min': -0.1, 'max': 0.8, 'palette': cm.palettes.RdYlGn}#cm.palettes.ndvi}

m.addLayer(img1, vis_params, "img1")
m.addLayer(img2, vis_params, "img2")
m.add_colorbar(vis_params, label="NDVI", layer_name="ndvi")
m.addLayer(aoi, {}, "aoi")
m.centerObject(aoi, 14)
m

In [ ]:
# df

In [ ]:
# df.loc[df.reducer.eq('mean')].plot.scatter(x='date', y='NDVI');

# Get Indices for each site

In [ ]:
outdir = LOCAL_DATA_DIR / "comparison_sites_cloud_20"
cloud_prob_thresh = 20

# Define remote sensing data source parameters
source_config = {
    "l7": {"source": landsat_7_sr, "start_date": dt.datetime(2000, 1, 1), "end_date": dt.datetime(2023, 5, 31), "scale": 30},
    "l8": {"source": landsat_8_sr, "start_date": dt.datetime(2013, 1, 1), "end_date": dt.datetime(2023, 5, 31), "scale": 30},
    "l9": {"source": landsat_9_sr, "start_date": dt.datetime(2021, 10, 31), "end_date": dt.datetime(2023, 5, 31), "scale": 30},
    "s2": {"source": s2_sr_harmonized, "start_date": dt.datetime(2018, 1, 1), "end_date": dt.datetime(2023, 5, 31), "scale": 10},
    # "s2": {"source": s2, "start_date": dt.datetime(2015, 6, 23), "end_date": dt.datetime(2023, 7, 31), "scale": 10},
}

# Mask for slope
mask = slope.lt(10)

# Set indices to calculate
indices = ['BI', 'EMBI', 'EVI', 'GLI', 'GNDVI', 'GVMI', 'MBI', 'MNLI', 'MSI', 'NDMI', 'NDVI', 'NGRDI', 'OSAVI', 'SAVI']

overwrite = True

outdir.mkdir(parents=True, exist_ok=True)

In [ ]:
# for source, config in source_config.items():
# # source = "s2"
# # config = source_config[source]
#     print(f"Downloading {indices} from source {source}")

#     fc = config["source"](
#         comparison_sites,
#         start_date,
#         end_date,
#         cloud_prob_thresh=cloud_prob_thresh
#     ).spectralIndices(indices)

#     # Apply mask to each image in the collection
#     fc_masked = fc.map(lambda img: img.updateMask(mask))

#     # Aggregate data by parcel
#     ts = fc_masked.getTimeSeriesByRegions(
#         reducer=[
#             ee.Reducer.mean(),
#             ee.Reducer.stdDev(),
#             ee.Reducer.min(),
#             ee.Reducer.max(),
#             ee.Reducer.count(),
#         ],
#         collection=comparison_sites,
#         bands=indices,
#         scale=config['scale'],
#         dateColumn="date",
#         naValue=None,
#     )

#     task = ee.batch.Export.table.toDrive(
#         collection=fc,
#         description=source,
#         folder='data',
#         fileNamePrefix=source,
#         fileFormat="CSV",
#     )

#     task.start()

In [ ]:
# n_properties = 4
# chunk_size = floor(4999 / n_properties)
# chunk_size

# n_chunks = ceil(4999 / chunk_size)
# n_chunks

In [ ]:
for source, config in source_config.items():
    if source != 's2': continue
    print(f"Downloading {indices} from source {source}")
    
    # Build lists of start and end dates so we extract yearly data - this is to overcome the limitation of 5000 elements with GEE
    start_date = config['start_date']
    end_date = config['end_date']
    start_dates = [start_date] + [dt.datetime(year, 1, 1) for year in range(start_date.year + 1, end_date.year + 1)]
    end_dates = [dt.datetime(year, 12, 31) for year in range(start_date.year, end_date.year)] + [end_date]
    
    for i, (start_date, end_date) in enumerate(tqdm(zip(start_dates, end_dates), total=len(start_dates))):
        
        filepath =  outdir / f"{source}_{start_date.year}.parquet"

        if filepath.exists():
            if not overwrite:
                continue

        fc = config["source"](
            comparison_sites,
            start_date,
            end_date,
            cloud_prob_thresh
        ).spectralIndices(indices)

        # Apply mask to each image in the collection
        fc_masked = fc.map(lambda img: img.updateMask(mask))

        # Aggregate data by parcel
        ts = fc_masked.getTimeSeriesByRegions(
            reducer=[
                ee.Reducer.mean(),
                ee.Reducer.stdDev(),
                ee.Reducer.min(),
                ee.Reducer.max(),
                ee.Reducer.count(),
            ],
            collection=comparison_sites,
            bands=indices,
            scale=config['scale'],
            dateColumn="date",
            naValue=None,
        )
        
        size = ts.size().getInfo()
        n_properties = len(ts.first().propertyNames().getInfo())
        
        if size * n_properties > 4999:
            chunk_size = floor(4999 / n_properties)
            n_chunks = ceil(4999 / chunk_size)

            df_list = []
            for j in range(n_chunks):
                offset = chunk_size * j
                try:
                    df = geemap.ee_to_pandas(ee.FeatureCollection(ts.toList(count=chunk_size, offset=offset)))
                    df_list.append(df)
                except:
                    pass
                df = pd.concat(df_list)
            df.to_parquet(filepath, index=None)
        else:
            df = geemap.ee_to_pandas(ts)
            df.to_parquet(filepath, index=None)

### Read local data

In [ ]:
# !ls -lhtr /Users/chill/software/bison-lab/data/local/comparison_sites_cloud_20

In [ ]:
df_list = []

for source, config in source_config.items():

    # generate dates for source
    start_date = config['start_date']
    end_date = config['end_date']
    start_dates = [start_date] + [dt.datetime(year, 1, 1) for year in range(start_date.year + 1, end_date.year + 1)]    

    for date in start_dates:
        filepath = outdir / f"{source}_{date.year}.parquet"
        if filepath.exists():
            df = pd.read_parquet(filepath)
            df["source"] = source
            df_list.append(df)
df = pd.concat(df_list)
df.date = pd.to_datetime(df.date)

# Drop NaN
df = df.dropna(subset=indices)

# Drop erroneous values and those that are likely too high or low
lower_limit = -0.05
upper_limit = 0.95

# for index in ['EVI', 'NDVI']:
for index in ['GLI', 'BI', 'SAVI', 'GVMI', 'NDVI', 'EMBI', 'OSAVI', 'NGRDI', 'MBI', 'EVI', 'MNLI', 'GNDVI']:
    include = (df[index] >= lower_limit) & (df[index] <= upper_limit)
    df.loc[~include, index] = np.nan

# melt to long form table
df_long = df.melt(
    id_vars=["date", "source", "name", "reducer"],
    value_vars=indices,
)

### Plot time series of parcel means

In [ ]:
# df_plot = df_long.loc[
#     df_long["reducer"].eq('mean')
#     & df_long["name"].eq('a_cattle')
#     & df_long["variable"].isin(['GLI', 'BI', 'SAVI', 'GVMI', 'NDVI', 'EMBI', 'OSAVI', 'NGRDI', 'MBI', 'EVI', 'MNLI', 'GNDVI'])
# ]

In [ ]:
# g = sns.relplot(
#     data=df_plot,
#     x="date",
#     y="value",
#     hue="name",
#     style="source",
#     col="variable",
#     col_wrap=1,
#     kind="scatter",
#     # ax=ax,
#     height=4,
#     aspect=4,
#     facet_kws=dict(sharey=False),
# )

# for ax in g.axes:
#     ax.xaxis.set_major_locator(mdates.YearLocator())
#     ax.xaxis.set_minor_locator(mdates.MonthLocator())

### Plot single parcel with errorbars

In [ ]:
ids = pd.IndexSlice

In [ ]:
index = "NDVI"
# names = ["a_cattle", "a_bison_7", "a_bison_7_1", "a_bison_7_2"]
names = ["a_cattle", "bison_7_green", "bison_7_reddish"]

df_index = df.copy().pivot(index=["date", "name"], columns="reducer", values=index)

n = len(names)
fig, ax = plt.subplots(nrows=n, figsize=(15, 4*n))
ax = ax.flatten()

for i, name in enumerate(names):
    a = df_index.loc[ids[:, name], :].reset_index(level="name", drop=True).sort_values("date")
    ax[i].errorbar(
        x=a.index,
        y=a["mean"],
        yerr=a["stdDev"],
        fmt=".",
        linewidth=0,
        elinewidth=0.5,
        color="k",
        capthick=0.5,
        capsize=1,
    )
    ax[i].set_title(f"{name} : {index}")

    ax[i].xaxis.set_major_locator(mdates.YearLocator())
    ax[i].xaxis.set_minor_locator(mdates.MonthLocator())
    ax[i].set_ylabel(f"{index}")
    ax[i].set_ylim((-0.4, 0.65))
fig.tight_layout()
# fig.savefig(local_data / f"{name}_ndvi.png", dpi=240)

In [ ]:
index = "EVI"
name = "c_bison_7"

df_index = df.copy().pivot(index=["date", "name"], columns="reducer", values=index)

fig, ax = plt.subplots(figsize=(15, 4))
a = df_index.loc[ids[:, name], :].reset_index(level="name", drop=True).sort_values("date")
ax.errorbar(
    x=a.index,
    y=a["mean"],
    yerr=a["stdDev"],
    fmt=".",
    linewidth=0,
    elinewidth=0.5,
    color="k",
    capthick=0.5,
    capsize=1,
)
ax.set_title(f"{name} : {index}")
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.set_ylabel(f"{index}")
ax.set_ylim((-0.4, 0.65))
fig.tight_layout()
# fig.savefig(local_data / f"{name}_ndvi.png", dpi=240)

In [ ]:
a = a[['mean']]

a['doy'] = a.index.day_of_year
a['year'] = a.index.year
a['month'] = a.index.month

b = a.groupby(['year', 'month']).mean()[['mean']].reset_index()

In [ ]:
df_plot = pd.pivot_table(b, index=b.month, columns=b.year, values='mean')
fig, ax = plt.subplots(figsize=(12,6));
df_plot[[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2008, 2009, 2010, 2011, 2012, 2013, 2014]].plot(ax=ax, marker='.', ls='-', alpha=1, color='b', legend=False)
df_plot[[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]].plot(ax=ax, marker='.', ls='-', alpha=1, color='r', legend=False);
fig.legend(loc='upper right', bbox_to_anchor=(1., 1),);

In [ ]:
df_plot = pd.pivot_table(b, index=b.month, columns=b.year, values='mean')
df_plot.plot(marker='.', ls='-', alpha=1, cmap='viridis', figsize=(12, 8)).legend(loc='upper right', bbox_to_anchor=(1.12, 1),);

In [ ]:
df_plot = pd.pivot_table(evi, index=evi.doy, columns=evi.year, values='mean')
df_plot.plot(marker='.', ls='none', alpha=1, cmap='viridis', figsize=(12, 8)).legend(loc='upper right', bbox_to_anchor=(1.12, 1),);

In [ ]:
# plot_names = ['a_cattle', 'a_bison_7', 'a_bison_7_1', 'a_bison_7_2']
plot_names = ['a_cattle', "bison_7_green", "bison_7_reddish"] #'a_bison_7_1', 'a_bison_7_2']
# plot_names = ['b_bison_7', 'b_cattle']
# plot_names = ['c_bison_7', 'c_bison_1']
# plot_names = ["bison_7_green", "bison_7_reddish"]

In [ ]:
df_plot = df_long.loc[
    df_long["reducer"].eq('mean')
    & (df_long["date"].dt.year >= 2010)
    & df_long["name"].isin(plot_names)
    # & df_long["name"].eq('a_bison_7')
    # & df_long["variable"].eq('NDVI')
    & df_long["variable"].isin(["NDVI", "EVI", "BI", "MBI", "SAVI"])
]

g = sns.relplot(
    data=df_plot,
    x="date",
    y="value",
    hue="name",
    style="source",
    col="variable",
    col_wrap=1,
    kind="scatter",
    # ax=ax,
    height=4,
    aspect=4,
    facet_kws=dict(sharey=False),
)

for ax in g.axes:
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.grid()
# g.savefig(LOCAL_DATA_DIR / f"a_indices_grid.png", dpi=240)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.transform import factor_cmap, factor_mark
output_notebook()

In [ ]:
markers = ['hex', 'circle_x', 'triangle']
source_names = ['l7', 'l8', 'l9', 's2']

p = figure(title="NDVI", background_fill_color="#fafafa", x_axis_type='datetime' )
p.xaxis.axis_label = 'date'
p.yaxis.axis_label = 'NDVI'
p.height = 600
p.width = 1500

p.scatter("date", "value", source=df_plot,
          legend_group="name", fill_alpha=0.5, size=8,
          marker=factor_mark('source', markers, source_names),
          color=factor_cmap('name', 'Category10_3', plot_names))

p.legend.location = "top_left"
show(p)

In [ ]:
ndvi = df_plot.pivot(columns="name", index="date", values="value")
# ndvi_monthly = ndvi.resample('1M').mean()

In [ ]:
ndvi_mean_monthly = ndvi.groupby(ndvi.index.month).mean()
# ndvi_mean_monthly = ndvi.groupby(ndvi.index.month).agg(['mean', 'std'])
ndvi_std_monthly = ndvi.groupby(ndvi.index.month).std()

In [ ]:
fig, ax = plt.subplots(figsize=(15,6));
ndvi_mean_monthly.plot(ax=ax, yerr=ndvi_std_monthly);

In [ ]:
ndvi_mean

In [ ]:
ndvi.plot(figsize=(15,6));

In [ ]:
ndvi_monthly.plot(figsize=(15,6));

In [ ]:
(ndvi_monthly.a_bison_7 - ndvi_monthly.a_cattle).mean()
# (ndvi_monthly.a_bison_7 - ndvi_monthly.b_cattle).mean()

In [ ]:
(ndvi_monthly.b_bison_7 - ndvi_monthly.b_cattle).describe()

In [ ]:
(ndvi_monthly.a_bison_7 - ndvi_monthly.a_cattle)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4));
((ndvi_monthly.a_bison_7 - ndvi_monthly.a_cattle) - (0.04818762778675109)).plot(ax=ax)
ax.axhline(y=0, color='k');

In [ ]:
(ndvi_monthly.a_bison_7 - ndvi_monthly.a_cattle).plot(figsize=(15, 4));

# Bison 7 year

In [ ]:
_a = kml_to_geodataframe(LOCAL_DATA_DIR / "Bison7_green_side.kml")

_b = kml_to_geodataframe(LOCAL_DATA_DIR / "Bison7-reddish_side.kml")

_b.loc[0, 'Name'] = "Bison7_reddish_side"

gdf = gpd.GeoDataFrame(pd.concat([_a, _b], ignore_index=True), crs=_a.crs)

bison_7_sites = geemap.geopandas_to_ee(gdf)

In [ ]:
bison_7_sites

In [ ]:
# m = geemap.Map(height=800)
# m.addLayer(bison_7_sites, {}, "")
# m.centerObject(bison_7_sites, 14)
# m

In [ ]:
outdir = LOCAL_DATA_DIR / "bison_7_cloud_20"
cloud_prob_thresh = 20

# Define remote sensing data source parameters
source_config = {
    "l7": {"source": landsat_7_sr, "start_date": dt.datetime(2000, 1, 1), "end_date": dt.datetime(2023, 5, 31), "scale": 30},
    "l8": {"source": landsat_8_sr, "start_date": dt.datetime(2013, 1, 1), "end_date": dt.datetime(2023, 5, 31), "scale": 30},
    "l9": {"source": landsat_9_sr, "start_date": dt.datetime(2021, 10, 31), "end_date": dt.datetime(2023, 5, 31), "scale": 30},
    "s2": {"source": s2_sr_harmonized, "start_date": dt.datetime(2018, 1, 1), "end_date": dt.datetime(2023, 5, 31), "scale": 10},
}

# Mask for slope
mask = slope.lt(10)

# Set indices to calculate
indices = ['BI', 'EMBI', 'EVI', 'GLI', 'GNDVI', 'GVMI', 'MBI', 'MNLI', 'MSI', 'NDMI', 'NDVI', 'NGRDI', 'OSAVI', 'SAVI']

overwrite = True

outdir.mkdir(parents=True, exist_ok=True)

In [ ]:
for source, config in source_config.items():
    print(f"Downloading {indices} from source {source}")
    
    # Build lists of start and end dates so we extract yearly data - this is to overcome the limitation of 5000 elements with GEE
    start_date = config['start_date']
    end_date = config['end_date']
    start_dates = [start_date] + [dt.datetime(year, 1, 1) for year in range(start_date.year + 1, end_date.year + 1)]
    end_dates = [dt.datetime(year, 12, 31) for year in range(start_date.year, end_date.year)] + [end_date]
    
    for i, (start_date, end_date) in enumerate(tqdm(zip(start_dates, end_dates), total=len(start_dates))):
        
        filepath =  outdir / f"{source}_{start_date.year}.parquet"

        if filepath.exists():
            if not overwrite:
                continue

        fc = config["source"](
            bison_7_sites,
            start_date,
            end_date,
            cloud_prob_thresh=cloud_prob_thresh
        ).spectralIndices(indices)

        # Apply mask to each image in the collection
        fc_masked = fc.map(lambda img: img.updateMask(mask))

        # Aggregate data by parcel
        ts = fc_masked.getTimeSeriesByRegions(
            reducer=[
                ee.Reducer.mean(),
                ee.Reducer.stdDev(),
                ee.Reducer.min(),
                ee.Reducer.max(),
                ee.Reducer.count(),
            ],
            collection=bison_7_sites,
            bands=indices,
            scale=config['scale'],
            dateColumn="date",
            naValue=None,
        )
        
#         size = ts.size().getInfo()
#         n_properties = len(ts.first().propertyNames().getInfo())
        
#         if size * n_properties > 4999:
#             chunk_size = floor(4999 / n_properties)
#             n_chunks = ceil(4999 / chunk_size)
            
#             df_list = []
#             for j in range(n_chunks):
#                 offset = chunk_size * j
#                 df = geemap.ee_to_pandas(ee.FeatureCollection(ts.toList(count=chunk_size, offset=offset)))
#                 df_list.append(df)
#             df = pd.concat(df_list)
#             df.to_parquet(filepath, index=None)
#         else:
        df = geemap.ee_to_pandas(ts)
        df.to_parquet(filepath, index=None)

In [ ]:
df_list = []

for source, config in source_config.items():

    # generate dates for source
    start_date = config['start_date']
    end_date = config['end_date']
    start_dates = [start_date] + [dt.datetime(year, 1, 1) for year in range(start_date.year + 1, end_date.year + 1)]    

    for date in start_dates:
        filepath = outdir / f"{source}_{date.year}.parquet"
        if filepath.exists():
            df = pd.read_parquet(filepath)
            df["source"] = source
            df_list.append(df)
df = pd.concat(df_list)
df.date = pd.to_datetime(df.date)

In [ ]:
# Drop NaN
df = df.dropna(subset=indices)

In [ ]:
# Drop erroneous values and those that are likely too high or low
lower_limit = -0.2
upper_limit = 0.95

# for index in ['EVI', 'NDVI']:
for index in indices:
# ['GLI', 'BI', 'SAVI', 'GVMI', 'NDVI', 'EMBI', 'OSAVI', 'NGRDI', 'MBI', 'EVI', 'MNLI', 'GNDVI']:
    include = (df[index] >= lower_limit) & (df[index] <= upper_limit)
    df.loc[~include, index] = np.nan

# melt to long form table
df_long = df.melt(
    id_vars=["date", "source", "Name", "reducer"],
    value_vars=indices,
)

In [ ]:
df_long

In [ ]:
# df_plot = df.loc[
#     (df["reducer"] == "mean")
#     # & (df["variable"] == "NDVI")
#     & df["name"].eq("a_bison_7")
# ][["date", "name", "NDVI", "source"]]

In [ ]:
# ndvi = df_plot.groupby('date')['NDVI'].mean()

In [ ]:
# fig, ax = plt.subplots(figsize=(12,6))
# sns.scatterplot(x="date", y="NDVI", data=df_plot, hue="source", ax=ax);

In [ ]:
# ndvi = ndvi.pivot(columns="name", index="date", values="value")
# ndvi_monthly = ndvi.resample('1M', label='right', closed='right').mean()

In [ ]:
# ndvi[['a_bison_1', 'a_bison_7', 'a_cattle']].plot(figsize=(15,6));

In [ ]:
# ndvi.iloc[(ndvi.index.year == 2023) & (ndvi.index.month ==5)].mean()

In [ ]:
# ndvi_monthly_plot = ndvi_monthly.melt(ignore_index=False).reset_index()
# ndvi_monthly_plot['doy'] = ndvi_monthly_plot.date.dt.day_of_year
# ndvi_monthly_plot['year'] = ndvi_monthly_plot.date.dt.year

In [ ]:
# ndvi_plot = ndvi.melt(ignore_index=False).reset_index()
# ndvi_plot['doy'] = ndvi_plot.date.dt.day_of_year
# ndvi_plot['year'] = ndvi_plot.date.dt.year

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 8));
# # g = sns.lineplot(data=ndvi_monthly_plot, x="doy", y="value", hue="year", style="name", estimator=None, lw=1, palette='viridis', ax=ax)
# g = sns.lineplot(data=ndvi_plot, x="doy", y="value", hue="year", style="name", estimator=None, lw=1, palette='viridis', ax=ax)

In [ ]:
# from scipy.signal import savgol_filter

# idx = (df_long.reducer == "mean") & (df_long.variable == "NDVI")
# df_long.loc[idx, 'value_smoothed'] = savgol_filter(df_long.loc[idx, 'value'], window_length=16, polyorder=3, deriv=0, delta=1.0, axis=-1, mode='interp', cval=None)

# idx = (df_long.reducer == "mean") & (df_long.variable == "EVI")
# df_long.loc[idx, 'value_smoothed'] = savgol_filter(df_long.loc[idx, 'value'], window_length=16, polyorder=3, deriv=0, delta=1.0, axis=-1, mode='interp', cval=None)

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 8));
# g = sns.lineplot(data=df_plot, x="doy", y="value", hue="year", style="source", estimator=None, lw=1, palette='viridis', ax=ax)
# # g = sns.lineplot(data=df_plot, x="doy", y="value_smoothed", hue="year", style="source", estimator=None, lw=1, palette='viridis', ax=ax)

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 8));
# g = sns.scatterplot(data=df_plot, x="doy", y="value", hue="year", style="source", palette='viridis', ax=ax)

In [ ]:
# df_plot = df_long.query("reducer =='mean' and date.dt.year >= 2015 and variable == 'NDVI'")